# Basic Setting

In [277]:
import matplotlib.pyplot as plt

import random
import numpy as np
import pandas as pd

import sympy
from sympy import Matrix, init_printing

from scipy.sparse.linalg import svds,eigs
from sklearn.preprocessing import MinMaxScaler

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances

from time import time
import warnings
warnings.filterwarnings(action='ignore')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ATK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [278]:
track = pd.read_csv('1160_global_track_spotify.csv')
track = pd.DataFrame(track)
# track = track.rename(columns={'Unnamed: 0':'index'})
# track_t = track.drop(['artist_name', 'track_name', 'album_name', ])
track.head()

,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,artist_popularity,track_popularity,artist_followers,danceability,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Harry's House,['pop'],94,92,21210921,167303,...,6,-5.338,0.0557,0.3420,0.001010,0.3110,0.662,173.930,167303,4
1,1,6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny,Me Porto Bonito,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,97,49472302,178567,...,1,-5.105,0.0817,0.0901,0.000027,0.0933,0.425,92.005,178567,4
2,2,3k3NWokhRRkEPhCzPmV8TW,Bad Bunny,Ojitos Lindos,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,97,49472302,258299,...,3,-5.745,0.0413,0.0800,0.000001,0.5280,0.268,79.928,258299,4
3,3,75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill (A Deal With God),Hounds Of Love,"['art pop', 'art rock', 'baroque pop', 'new wa...",76,89,1046980,298933,...,10,-13.123,0.0550,0.7200,0.003140,0.0604,0.197,108.375,298933,4
4,4,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,95,49472302,243717,...,5,-5.198,0.2530,0.0993,0.000291,0.1260,0.187,106.672,243717,4


# Feature Engineering 

In [279]:
import re
 
def cleanText(readData):
    text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData) # 쉼표(,) 뺌
 
    return text

In [280]:
genre = []
for i in track['artist_genre']:
    if i == '[]':
        i = 'NA'
        genre.append(i.strip()) #"'[]'"
    else:
        i = cleanText(i)
        genre.append(i.strip())
track['genre'] = genre

In [281]:
track['track_popularity'] = track['track_popularity'] / 100 

In [282]:
track.head()

,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,artist_popularity,track_popularity,artist_followers,danceability,...,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,0,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Harry's House,['pop'],94,0.92,21210921,167303,...,-5.338,0.0557,0.3420,0.001010,0.3110,0.662,173.930,167303,4,pop
1,1,6Sq7ltF9Qa7SNFBsV5Cogx,Bad Bunny,Me Porto Bonito,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.97,49472302,178567,...,-5.105,0.0817,0.0901,0.000027,0.0933,0.425,92.005,178567,4,"latin, reggaeton, trap latino"
2,2,3k3NWokhRRkEPhCzPmV8TW,Bad Bunny,Ojitos Lindos,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.97,49472302,258299,...,-5.745,0.0413,0.0800,0.000001,0.5280,0.268,79.928,258299,4,"latin, reggaeton, trap latino"
3,3,75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill (A Deal With God),Hounds Of Love,"['art pop', 'art rock', 'baroque pop', 'new wa...",76,0.89,1046980,298933,...,-13.123,0.0550,0.7200,0.003140,0.0604,0.197,108.375,298933,4,"art pop, art rock, baroque pop, new wave pop, ..."
4,4,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Un Verano Sin Ti,"['latin', 'reggaeton', 'trap latino']",100,0.95,49472302,243717,...,-5.198,0.2530,0.0993,0.000291,0.1260,0.187,106.672,243717,4,"latin, reggaeton, trap latino"


## Collecting User info

In [283]:
# import random
# songs = list(track['track_name'].values)
# song = random.sample(songs, 5)

# total_dictionary = {}
# qs=[]
# qs.append("무슨 곡이 가장 좋아요?   1) {}  2) {}  3) {}  4) {}  5) {}".format(song[0],song[1],song[2],song[3],song[4]))
# qs.append("어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운")
# qs.append("어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림")
# qs.append("고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔")
# qs.append("끝")


# for q in qs:
#     question = q
#     if question == "끝":
#         break
#     else:
#         total_dictionary[question] = ""

# for i in total_dictionary:
#     print(i)
#     answer = input()
#     total_dictionary[i] = answer 

# a = list(total_dictionary.items())
# answer_1 = a[0][1]
# answer_2 = a[1][1]
# answer_3 = a[2][1]
# answer_4 = a[3][1]

## Genre Recommeder

In [413]:
def recommend_genre(data, music_name, top=200):

    # TF-IDF
    tfidf = TfidfVectorizer(ngram_range=(1,2))
    tf_genre = tfidf.fit_transform(data.genre)

    # 코사인 유사도
    ts_genre = cosine_similarity(tf_genre, tf_genre)

    #특정 장르 정보 뽑아오기
    target_genre_index = data[data['track_name'] == music_name].index.values

    # 입력한 영화의 유사도 데이터 프레임 추가
    data["cos_similarity"] = ts_genre[target_genre_index, :].reshape(-1,1)
    sim_genre_index = data[data.index != target_genre_index[0]].index
    sim_genre = data.iloc[sim_genre_index].sort_values(by="cos_similarity", ascending=False)
    final_index = sim_genre.index.values[ : top]
    result_genre = data.iloc[final_index]
    
    return result_genre[['artist_name', 'track_name', 'cos_similarity']]

## Genre + Artist/Track/Album name Recommeder

In [378]:
# def recommend_genre(data, music_name, top=200):

#     # TF-IDF
#     tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2) ,stop_words=stopwords.words('english'))
#     tf_genre = tfidf.fit_transform(data['artist_name'] + ' ' +
#                                    data['track_name'] + ' ' +
#                                    data['album_name'] + ' ' +
#                                    data['genre'])

#     # 코사인 유사도
#     ts_genre = cosine_similarity(tf_genre, tf_genre)

#     #특정 장르 정보 뽑아오기
#     target_genre_index = data[data['track_name'] == music_name].index.values

#     # 입력한 영화의 유사도 데이터 프레임 추가
#     data["cos_similarity"] = ts_genre[target_genre_index, :].reshape(-1,1)
#     sim_genre_index = track[track.index != target_genre_index[0]].index
#     sim_genre = data.iloc[sim_genre_index].sort_values(by="cos_similarity", ascending=False)
#     final_index = sim_genre.index.values[ : top]
#     result_genre = data.iloc[final_index]
    
#     return result_genre[['artist_name', 'track_name', 'cos_similarity']]

## Features Recommender

In [415]:
def recommend_features(track, x, top=200):
    
    scaler = MinMaxScaler()
    index = track[track['track_name'] == x].index.values
    track1 = track[['danceability', 'energy', 'valence', 'tempo', 'acousticness']]
    track_scaled = scaler.fit_transform(track1)
    target_index = track_scaled[index]
    
    euclidean = []
    for value in track_scaled:
        eu = euclidean_distance(target_index, value)
        euclidean.append(eu)
    
    track['euclidean_distance'] = euclidean
    sim_feature_index = track[track.index != index[0]].index

    result_feature = track.iloc[sim_feature_index].sort_values(by='euclidean_distance', ascending=True)[:200]
#     result = track.iloc[sim_feature_index][:10]


    return result_feature[['artist_name', 'track_name', 'euclidean_distance']]

## Intersection of Genre & Feature

In [ ]:
def feature_genre_intersection(a, b):
    intersected = pd.merge(a, b, how='inner')
    similarity = intersected[['euclidean_distance', 'cos_similarity']]
    temp =scaler.fit_transform(similarity)
    temp = pd.DataFrame(temp, columns = ['euclidean_scaled', 'cosine_scaled'])

    intersected['euclidean_scaled'] = temp['euclidean_scaled']
    intersected['cosine_scaled'] = temp['cosine_scaled']

    intersected['ratio'] = intersected['euclidean_scaled'] + (1 - intersected['cosine_scaled'])
    result = intersected.sort_values('ratio', ascending=True)[:10]
    return result

# Content-Based Recommeder
- When a User first signs up on website

In [530]:
class ContentBasedRecommender:
    
    def __init__(self, data):
        self.data = data
        self.music = ''
        self.mood = ''
        self.speed = ''
        self.emotion = ''

        
    def user_info(self):
        print("--------------------------------------------------------------------\n\
노래를 추천해드리기 전에 잠시 당신에 대해서 알아보겠습니다 \n\
--------------------------------------------------------------------")
        songs = list(self.data['track_name'].values)
        song = random.sample(songs, 5)

        total_dictionary = {}
        qs=[]
        qs.append("무슨 곡이 가장 좋아요?   1) {}  2) {}  3) {}  4) {}  5) {}".format(song[0],song[1],song[2],song[3],song[4]))
        qs.append("어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운")
        qs.append("어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림")
        qs.append("고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔")
        qs.append("끝")

        for q in qs:
            question = q
            if question == "끝":
                break
            else:
                total_dictionary[question] = ""

        for i in total_dictionary:
            print(i)
            answer = input()
            total_dictionary[i] = answer 

        a = list(total_dictionary.items())
        self.music = a[0][1]
        self.mood = a[1][1]
        self.speed = a[2][1]
        self.emotion = a[3][1]
        
        return [self.music, self.mood, self.speed, self.emotion]

    
    def euclidean_distance(x, y):   
        
        return np.sqrt(np.sum((x - y) ** 2))
    
    
    def recommend_features(self, music_name, top=200):
    
        scaler = MinMaxScaler()
        index = self.data[self.data['track_name'] == music_name].index.values
        track_new = self.data[['danceability', 'energy', 'valence', 'tempo', 'acousticness']]
        track_scaled = scaler.fit_transform(track_new)
        target_index = track_scaled[index]

        euclidean = []
        for value in track_scaled:
            eu = euclidean_distance(target_index, value)
            euclidean.append(eu)

        self.data['euclidean_distance'] = euclidean
        sim_feature_index = self.data[self.data.index != index[0]].index

        result_features = self.data.iloc[sim_feature_index].sort_values(by='euclidean_distance', ascending=True)[:top]
    #     result = track.iloc[sim_feature_index][:10]

        return result_features[['artist_name', 'track_name', 'euclidean_distance']]

    
    def recommend_genre(self, music_name, top=200):
        
        # TF-IDF
        tfidf = TfidfVectorizer(ngram_range=(1,2))
        tf_genre = tfidf.fit_transform(self.data.genre)

        # 코사인 유사도
        ts_genre = cosine_similarity(tf_genre, tf_genre)

        #특정 장르 정보 뽑아오기
        target_genre_index = self.data[self.data['track_name'] == music_name].index.values

        # 입력한 영화의 유사도 데이터 프레임 추가
        self.data["cos_similarity"] = ts_genre[target_genre_index, :].reshape(-1,1)
        sim_genre_index = self.data[self.data.index != target_genre_index[0]].index
        sim_genre = self.data.iloc[sim_genre_index].sort_values(by="cos_similarity", ascending=False)
        final_index = sim_genre.index.values[ : top]
        result_genre = self.data.iloc[final_index]

        return result_genre[['artist_name', 'track_name', 'cos_similarity']]

    
    def feature_genre_intersection(self, recommended_feature, recommended_genre, top=10):
        
        print("--------------------------------------------------------------\n\
장르 / 노래 분위기 / 노래 속도 / User 기분상태에 따라 추천을 해드리겠습니다 \n\
--------------------------------------------------------------")
        
        intersection = pd.merge(recommended_feature, recommended_genre, how='inner')
        similarity = intersection[['euclidean_distance', 'cos_similarity']]
        scale = scaler.fit_transform(similarity)
        scale = pd.DataFrame(scale, columns=['eu_scaled', 'cos_scaled'])
        
        intersection['euclidean_scaled'] = scale['eu_scaled']
        intersection['cosine_scaled'] = scale['cos_scaled']
        intersection['ratio'] = intersection['euclidean_scaled'] + (1 - intersection['cosine_scaled'])
        result_intersection = intersection.sort_values('ratio', ascending=True)
        
        return result_intersection[:top]

    
a = ContentBasedRecommender(track)
b = a.user_info()
c = a.recommend_features(b[0])
d = a.recommend_genre(b[0])
e = a.feature_genre_intersection(c,d)
e


--------------------------------------------------------------------
노래를 추천해드리기 전에 잠시 당신에 대해서 알아보겠습니다 
--------------------------------------------------------------------
무슨 곡이 가장 좋아요?   1) a car going nowhere  2) 31  3) Until I Found You  4) ONLY LOOK AT ME - KR Ver.  5) December
고해
어떤 분위기의 곡을 좋아하세요?   1) 밝은  2) 어두운
ㅁ
어느정도의 속도를 가직 곡을 좋아하시나요?   1) 빠름  2) 느림
ㅁ
고객님께서는 현재 감정이 어떤 상태이신가요?   1) 신남  2) 평범  3) 슬픔
a
--------------------------------------------------------------
장르 / 노래 분위기 / 노래 속도 / User 기분상태에 따라 추천을 해드리겠습니다 
--------------------------------------------------------------


,artist_name,track_name,euclidean_distance,cos_similarity,euclidean_scaled,cosine_scaled,ratio
5,SG Wannabe,Love You,0.366380,0.945366,0.611710,1.000000,0.611710
7,SG Wannabe,라라라,0.381104,0.945366,0.666258,1.000000,0.666258
4,Yoon Mirae,"Get It In (feat.Tiger JK, Jung In) (Korean Ver.)",0.362596,0.825802,0.597690,0.831491,0.766199
0,Gummy,혼자만 하는 사랑,0.201266,0.350723,0.000000,0.161932,0.838068
16,SG Wannabe,Must Have Love,0.445887,0.945366,0.906265,1.000000,0.906265
18,Yangpa,A'ddio,0.451144,0.777266,0.925741,0.763086,1.162654
1,AKMU,Last Goodbye,0.298247,0.350723,0.359292,0.161932,1.197361
3,BROWN EYED SOUL,My Everything,0.351871,0.484091,0.557957,0.349896,1.208061
11,DICKPUNKS,1,0.423347,0.484091,0.822760,0.349896,1.472863
2,Lee Soo Young,Grace,0.339591,0.247592,0.512463,0.016583,1.495880


In [461]:
def softmax_(x):
    c = np.max(x)
    exp_x = np.exp(x - c)
    sum_exp_x = np.sum(exp_x)
    
    return exp_x / sum_exp_x

In [97]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [513]:
a = recommend_features(track,"이럴거면 그러지말지 Shouldn't Have… (feat. Young K)")
# a = a[['artist_name', 'track_name']]
b = recommend_genre(track, "이럴거면 그러지말지 Shouldn't Have… (feat. Young K)")
# b = b[['artist_name', 'track_name']]

In [134]:
track_scaled = track[['danceability', 'energy', 'valence', 'tempo', 'acousticness']]

track_scaled = scaler.fit_transform(track_scaled)
print(track_scaled[188])
print(track_scaled[412])
euclidean_distance(track_scaled[188],track_scaled[412])

[0.37197556 0.82582583 0.24897119 0.45674436 0.00088028]
[0.45924854 0.67867868 0.14300412 0.36076655 0.05140845]


0.22861039718992332

# Content-Based Recommender 
- When website does not have enough users

# Collaborative Filtering Recommender
- When there are enough users